In [193]:
# transform str feature in to values so that it can be use in the model 
import re
import pandas as pd
import os
def feature_tovalue (input_file, output_file):
# this function is to transform some features in to a value form for the modelling later such as the bus train tram distance , schoolr rank
# such as shopping center distance 
# input_file: the csv file needed to add in more features 
# output_file: is the csv name and the output path of the merged/adjusted csv file
    bus_distance_value_list=[]
    tram_distance_value_list=[]
    train_distance_value_list=[]
    country_train_distance_value_list=[]
    df = pd.read_csv(input_file)
    school_rank_value_list =[]
    walk_time_list =[]
    drive_time_list =[]
 
    # clean the data first 
    new_column_names = {'crime_rate_per 100,000 population': 'Crime_rate', 'Population_density': 'Pop_density'}
    df = df.rename(columns=new_column_names)
    df['cost_text'] = pd.to_numeric(df['cost_text'], errors='coerce')
    
    df = df[(df['Beds'] != 0) & 
                (df['Baths'] != 0) & 
                (df['cost_text']>50)
                ] 
    df = df.dropna( subset=['SA2_code'])
    df = df.reset_index(drop=True)
    
    # Add transport value 
    for count in range (len(df)):
        # add school rank value 
        # for each property nearby schools 
        top_rank = 650
        for i in df["School_with_rank"][count].split("],"):
            
            string=  i.split(",")[2]
            pattern = r"(?:None|\d+)"
            matches = re.findall(pattern, string)
            # to get the nearby school rank 
            if matches[0]!="None":
                rank = int(matches[0])
                if rank<top_rank:
                    top_rank = rank
        
        school_rank_value_list.append(top_rank/650)
        
        
        walk_time = 8200
        drive_time=1100
        if not pd.isna(df["driving distance/time"][count]):
            drive_time= df["driving distance/time"][count].split("/")[1]
            walk_time=df["walking distance/time"][count].split("/")[1]
        
        walk_time_list.append(walk_time)
        drive_time_list.append(drive_time)
        
        try:
            pattern = r'\d+\.\d+'
            match = re.search(pattern, df["Nearest_Bus_Stop"].iloc[count].split(",")[3])
            
            bus_distance_value  = float(float(match.group(0))/3)
            bus_distance_value_list.append(bus_distance_value)
        except AttributeError :
            bus_distance_value = 1
            bus_distance_value_list.append(bus_distance_value)
        
        try:
            match = re.search(pattern, df["Nearest_Tram_Stop"].iloc[count].split(",")[3])
            tram_distance_value  = float(float(match.group(0))/5)
            tram_distance_value_list.append(tram_distance_value)
        except AttributeError :
            tram_distance_value = 1
            tram_distance_value_list.append(tram_distance_value)
            
        try:
            match = re.search(pattern, df["Nearest_Metro_Train_Station"].iloc[count].split(",")[3])
            train_distance_value  = float(float(match.group(0))/10)
            train_distance_value_list.append(train_distance_value)
        except AttributeError :
            train_distance_value = 1
            train_distance_value_list.append(train_distance_value)
        try:
            match = re.search(pattern, df["Nearest_Regional_Train_Station"].iloc[count].split(",")[3])
            country_train_distance_value  = float(float(match.group(0))/30)
            country_train_distance_value_list.append(country_train_distance_value)
        except AttributeError :
            country_train_distance_value = 1
            country_train_distance_value_list.append(country_train_distance_value)
            
    
    df["school_rank_value"] = school_rank_value_list 
    df["bus_value"]=bus_distance_value_list
    df["tram_value"]= tram_distance_value_list
    df["train_value"]=train_distance_value_list
    df["country_train"]= country_train_distance_value_list
    df["walk_time"]=walk_time_list
    df["drive_time"]=drive_time_list
    if not os.path.exists("../data/curated/twelveth_clean"):
        os.makedirs("../data/curated/twelveth_clean")
            
    df.to_csv(output_file,index = False )
        
        


In [194]:
feature_tovalue("../data/curated/eleventh_clean/apartment_1.csv","../data/curated/twelveth_clean/apartment_1.csv")
feature_tovalue("../data/curated/eleventh_clean/apartment_2.csv","../data/curated/twelveth_clean/apartment_2.csv")
feature_tovalue("../data/curated/eleventh_clean/apartment_3.csv","../data/curated/twelveth_clean/apartment_3.csv")
feature_tovalue("../data/curated/eleventh_clean/apartment_4.csv","../data/curated/twelveth_clean/apartment_4.csv")
feature_tovalue("../data/curated/eleventh_clean/apartment_5.csv","../data/curated/twelveth_clean/apartment_5.csv")

feature_tovalue("../data/curated/eleventh_clean/house_1.csv","../data/curated/twelveth_clean/house_1.csv")
feature_tovalue("../data/curated/eleventh_clean/house_2.csv","../data/curated/twelveth_clean/house_2.csv")
feature_tovalue("../data/curated/eleventh_clean/house_3.csv","../data/curated/twelveth_clean/house_3.csv")
feature_tovalue("../data/curated/eleventh_clean/house_4.csv","../data/curated/twelveth_clean/house_4.csv")

feature_tovalue("../data/curated/eleventh_clean/town_house.csv","../data/curated/twelveth_clean/town_house.csv")


In [196]:
# this part is not a function it combines all the apartment csv and store it in the ../data/cruated/final_clean 
if not os.path.exists("../data/curated/final_data"):
    os.makedirs("../data/curated/final_data")
dfs = []
file_paths = ["../data/curated/twelveth_clean/apartment_1.csv", "../data/curated/twelveth_clean/apartment_2.csv", "../data/curated/twelveth_clean/apartment_3.csv",
              "../data/curated/twelveth_clean/apartment_4.csv", "../data/curated/twelveth_clean/apartment_5.csv"]  # Add the file paths for your CSV files

for file_path in file_paths:
    df = pd.read_csv(file_path)
    dfs.append(df)
merged_df_apartment = pd.concat(dfs, ignore_index=True)
merged_df_apartment.to_csv("../data/curated/final_data/apartment.csv")

In [197]:
# this part is not a function it combines all the house csv and store it in the ../data/cruated/final_clean 
dfs = []
file_paths = ["../data/curated/twelveth_clean/house_1.csv", "../data/curated/twelveth_clean/house_3.csv",
              "../data/curated/twelveth_clean/house_2.csv", "../data/curated/twelveth_clean/house_4.csv"]  # Add the file paths for your CSV files

for file_path in file_paths:
    df = pd.read_csv(file_path)
    dfs.append(df)
merged_df_house = pd.concat(dfs, ignore_index=True)
merged_df_house.to_csv("../data/curated/final_data/house.csv")

In [198]:
# this part is not a function it combines all the townhouse csv and store it in the ../data/cruated/final_clean 
df = pd.read_csv("../data/curated/twelveth_clean/town_house.csv")
df.to_csv("../data/curated/final_data/town_house.csv")